In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import time
import datetime
import os,gc
file_path='D:/360WiFi/'
raw_data_path='D:/360WiFi/FDDC_financial_data_20180613/'
split_file_path='D:/360WiFi/files/financial_data_new/'
'''
Function List:

    loadingFile:读取原始文件
    loading_12_file:读取12表
    date_time：将表中三日期datatime化,并排序
    date_time_index：date_time排序的时候加上INDEX
    merge_3sheet：合成三大表
    ticker_3sheet_info：检查上市后ticker是否有三表缺失情况
    check_uniqueness：检查三DATE联立后是否有冲突数据
    fill_nan_PIT:将12表的point in time 最新数据缺失值补全
    
    preprocess:合三表（涉及到）preprocess_common，preprocess_normal，preprocess_special，
                              其中preprocess_common又涉及到loading_inputs

'''



def loadingFile(whether_code):
    '''
    whether_code=1,替换columns
    whether_code=0，不替换columns
    
    
    return 12 original files 
    use time: 25mins
    '''
    #------处理 BalanceSheet---------------------
    
    start_time=time.time()
    balance_sheet_Bank = pd.read_excel(raw_data_path+'Balance Sheet.xls',sheetname='Bank')
    balance_sheet_Securities = pd.read_excel(raw_data_path+'Balance Sheet.xls',sheetname='Securities')
    balance_sheet_Insurance = pd.read_excel(raw_data_path+'Balance Sheet.xls',sheetname='Insurance')
    balance_sheet_GB = pd.read_excel(raw_data_path+'Balance Sheet.xls',sheetname='General Business')

    print('Reading Balance sheet use time:{}\n'.format(time.time()-start_time))


    #处理---------- Cash Flow Statement--------------------
    
    start_time=time.time()
    cashFlow_sheet_Bank = pd.read_excel(raw_data_path+'Cash Flow Statement.xls',sheetname='Bank')
    cashFlow_sheet_Securities = pd.read_excel(raw_data_path+'Cash Flow Statement.xls',sheetname='Securities')
    cashFlow_sheet_Insurance = pd.read_excel(raw_data_path+'Cash Flow Statement.xls',sheetname='Insurance')
    cashFlow_sheet_GB = pd.read_excel(raw_data_path+'Cash Flow Statement.xls',sheetname='General Business')
    print('Reading Cash Flow sheet use time:{}\n'.format(time.time()-start_time))
    cashFlow_sheet_GB


    #处理 ----------Income Statement------------------------

    start_time=time.time()
    income_sheet_Bank = pd.read_excel(raw_data_path+'Income Statement.xls',sheetname='Bank')
    income_sheet_Securities = pd.read_excel(raw_data_path+'Income Statement.xls',sheetname='Securities')
    income_sheet_Insurance = pd.read_excel(raw_data_path+'Income Statement.xls',sheetname='Insurance')
    income_sheet_GB = pd.read_excel(raw_data_path+'Income Statement.xls',sheetname='General Business')
    print('Reading Cash Flow sheet use time:{}\n'.format(time.time()-start_time))
    income_sheet_GB


    #-----------更名操作---------------------
    '''
    将三表中的一般工商、银行、保险、证券的科目具体标注，方便查看
    sample as blew:
    # balance_bank_old_columns=list(balance_sheet_Bank.columns[9:])
    # balance_bank_new_columns=[x+'_balance_bank' for x in balance_bank_old_columns]
    # balance_bank_columns_dic=dict(zip(balance_bank_old_columns,balance_bank_new_columns))
    # balance_sheet_Bank.rename(columns=balance_bank_columns_dic,inplace=True)
    '''
    if whether_code==1:
    
        target_df=[balance_sheet_Bank,balance_sheet_Securities, balance_sheet_Insurance,balance_sheet_GB,\
                   cashFlow_sheet_Bank,cashFlow_sheet_Securities,cashFlow_sheet_Insurance,cashFlow_sheet_GB,\
                   income_sheet_Bank,income_sheet_Securities,income_sheet_Insurance,income_sheet_GB]
        new_name=['_balance_Bank','_balance_Securities','_balance_Insurance','_balance_GB',\
                    '_cashFlow_Bank','_cashFlow_Securities','_cashFlow_Insurance','_cashFlow_GB',\
                  '_income_Bank','_income_Securities','_income_Insurance','_income_GB']
        for (item,name) in zip(target_df,new_name):
            old_columns=list(item.columns[9:])
            new_columns=[x+name for x in old_columns]
            columns_dic=dict(zip(old_columns,new_columns))
            item.rename(columns=columns_dic,inplace=True)

    
    
    
    return balance_sheet_Bank,balance_sheet_Securities, balance_sheet_Insurance,balance_sheet_GB,\
            cashFlow_sheet_Bank,cashFlow_sheet_Securities,cashFlow_sheet_Insurance,cashFlow_sheet_GB,\
           income_sheet_Bank,income_sheet_Securities,income_sheet_Insurance,income_sheet_GB

##--------读取12表--------------------------
def loading_12_file():
    '''
    读取12张表
    '''
    balance_sheet_Bank_old=pd.read_csv(file_path+'files/financial_data_new/balance_sheet_Bank.csv')
    balance_sheet_Securities_old=pd.read_csv(file_path+'files/financial_data_new/balance_sheet_Securities.csv')
    balance_sheet_Insurance_old=pd.read_csv(file_path+'files/financial_data_new/balance_sheet_Insurance.csv')
    balance_sheet_GB_old=pd.read_csv(file_path+'files/financial_data_new/balance_sheet_GB.csv')

    cashFlow_sheet_Bank_old=pd.read_csv(file_path+'files/financial_data_new/cashFlow_sheet_Bank.csv')
    cashFlow_sheet_Securities_old=pd.read_csv(file_path+'files/financial_data_new/cashFlow_sheet_Securities.csv')
    cashFlow_sheet_Insurance_old=pd.read_csv(file_path+'files/financial_data_new/cashFlow_sheet_Insurance.csv')
    cashFlow_sheet_GB_old=pd.read_csv(file_path+'files/financial_data_new/cashFlow_sheet_GB.csv')

    income_sheet_Bank_old=pd.read_csv(file_path+'files/financial_data_new/income_sheet_Bank.csv')
    income_sheet_Securities_old=pd.read_csv(file_path+'files/financial_data_new/income_sheet_Securities.csv')
    income_sheet_Insurance_old=pd.read_csv(file_path+'files/financial_data_new/income_sheet_Insurance.csv')
    income_sheet_GB_old=pd.read_csv(file_path+'files/financial_data_new/income_sheet_GB.csv')
    
    return balance_sheet_Bank_old,balance_sheet_Securities_old,balance_sheet_Insurance_old,balance_sheet_GB_old,\
            cashFlow_sheet_Bank_old,cashFlow_sheet_Securities_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_GB_old,\
            income_sheet_Bank_old,income_sheet_Securities_old,income_sheet_Insurance_old,income_sheet_GB_old




def date_time(df):
    df['END_DATE']=pd.to_datetime(df['END_DATE'],format='%Y/%m/%d')
    df['PUBLISH_DATE']=pd.to_datetime(df['PUBLISH_DATE'],format='%Y/%m/%d')
    df['END_DATE_REP']=pd.to_datetime(df['END_DATE_REP'],format='%Y/%m/%d')
    df.sort_values(by=['TICKER_SYMBOL','END_DATE','PUBLISH_DATE','END_DATE_REP'],inplace=True)
    
    return df


def merge_3sheet():
    
    '''
    将12表，四四合成，形成三表，并做point_in_time处理
    
    return  合好的三表【仅有'TICKER_SYMBOL','PUBLISH_DATE','END_DATE_REP','END_DATE','MERGED_FLAG'这几列】
    '''

    balance_sheet_Bank,balance_sheet_Securities, balance_sheet_Insurance,balance_sheet_GB,\
    cashFlow_sheet_Bank,cashFlow_sheet_Securities,cashFlow_sheet_Insurance,cashFlow_sheet_GB,\
    income_sheet_Bank,income_sheet_Securities,income_sheet_Insurance,income_sheet_GB=loading_12_file()

    common_list=['TICKER_SYMBOL','PUBLISH_DATE','END_DATE_REP','END_DATE','MERGED_FLAG']

    merge_list_balance=[balance_sheet_Bank[common_list],balance_sheet_Securities[common_list], balance_sheet_Insurance[common_list],balance_sheet_GB[common_list]]
    merge_list_cashFlow=[cashFlow_sheet_Bank[common_list],cashFlow_sheet_Securities[common_list],cashFlow_sheet_Insurance[common_list],cashFlow_sheet_GB[common_list]]
    merge_list_income=[income_sheet_Bank[common_list],income_sheet_Securities[common_list],income_sheet_Insurance[common_list],income_sheet_GB[common_list]]


    Merge_Balance=pd.concat(merge_list_balance)
    Merge_CashFlow=pd.concat(merge_list_cashFlow)
    Merge_Income=pd.concat(merge_list_income)

    Merge_Balance=date_time(Merge_Balance)
    Merge_CashFlow=date_time(Merge_CashFlow)
    Merge_Income=date_time(Merge_Income)

    Merge_Balance.drop_duplicates(subset=['TICKER_SYMBOL','END_DATE'], keep='last',inplace=True)
    Merge_CashFlow.drop_duplicates(subset=['TICKER_SYMBOL','END_DATE'], keep='last',inplace=True)
    Merge_Income.drop_duplicates(subset=['TICKER_SYMBOL','END_DATE'], keep='last',inplace=True)
    
    return Merge_Balance,Merge_CashFlow,Merge_Income



    ##check ticker的三表情况
    
#生成基准上市之后的ticker-时间 df

def ticker_3sheet_info(Merge_Balance,Merge_CashFlow,Merge_Income):
    '''
    输入是合好并且去重的三表，生成上市公司三表统计，注意有30个ticker缺乏上市时间，未在列
    
    
    [  67, 1043, 1226, 1236, 1426, 1796, 1888, 2074, 2151, 2164, 2165,
            2166, 2167, 2168, 3001, 3013, 3028, 3122, 3150, 3238, 3252, 3253,
            3280, 3314, 3351, 3355, 3426, 3434, 3478, 3494]
    
    
    '''

    list_date=pd.read_csv(file_path+'ticker_market.csv',encoding='gbk')
    list_date.dropna(subset=['listDate'],inplace=True)
    #list_date['TICKER_SYMBOL']=list_date['TICKER_SYMBOL'].apply(lambda x:"%06d"%x)
    list_date_dic=dict(zip(list_date.TICKER_SYMBOL,list_date.listDate))
    list_date_dic

    range_list=['2009-03-31', '2009-06-30', '2009-09-30', '2009-12-31',\
        '2010-03-31', '2010-06-30', '2010-09-30', '2010-12-31',\
        '2011-03-31', '2011-06-30', '2011-09-30', '2011-12-31',\
        '2012-03-31', '2012-06-30', '2012-09-30', '2012-12-31',\
        '2013-03-31', '2013-06-30', '2013-09-30', '2013-12-31',\
        '2014-03-31', '2014-06-30', '2014-09-30', '2014-12-31',\
        '2015-03-31', '2015-06-30', '2015-09-30', '2015-12-31',\
        '2016-03-31', '2016-06-30', '2016-09-30', '2016-12-31',\
        '2017-03-31', '2017-06-30', '2017-09-30', '2017-12-31',\
        '2018-03-31']


    for ticker in list_date_dic.keys():
        for i in range(len(range_list)-1):
            if list_date_dic[ticker]<=range_list[0]:
                list_date_dic[ticker]=range_list[0]
                break
            elif (list_date_dic[ticker]>range_list[i])&(list_date_dic[ticker]<=range_list[i+1]):
                list_date_dic[ticker]=range_list[i+1]
    list_date_dic


    date_dic=dict()
    for i in range(len(range_list)):
        date_dic[range_list[i]]=range_list[i:]
    date_dic
    list_date_dic


    DATE=[]
    TICKER=[]
    for ticker in list_date_dic.keys():
        TICKER.extend([ticker for x in range(len(date_dic[list_date_dic[ticker]]))])
        DATE.extend(date_dic[list_date_dic[ticker]])

    BASE_list=DataFrame({'TICKER_SYMBOL':TICKER,'END_DATE':DATE})
    BASE_list['END_DATE']=pd.to_datetime(BASE_list['END_DATE'],format='%Y/%m/%d')
    BASE_list


    # 将三表分别merge基准df中

    Merge_Balance.rename(columns={'MERGED_FLAG':'Balance'},inplace=True)
    Merge_CashFlow.rename(columns={'MERGED_FLAG':'CashFlow'},inplace=True)
    Merge_Income.rename(columns={'MERGED_FLAG':'Income'},inplace=True)

    BASE1=pd.merge(BASE_list,Merge_Balance[['TICKER_SYMBOL','END_DATE','Balance']],how='left',on=['TICKER_SYMBOL','END_DATE'])
    BASE2=pd.merge(BASE1,Merge_CashFlow[['TICKER_SYMBOL','END_DATE','CashFlow']],how='left',on=['TICKER_SYMBOL','END_DATE'])
    BASE3=pd.merge(BASE2,Merge_Income[['TICKER_SYMBOL','END_DATE','Income']],how='left',on=['TICKER_SYMBOL','END_DATE'])
    BASE3.to_csv(file_path+'上市公司_三表统计.csv',index=False)
    
    pass



def check_uniqueness():
    
    '''
   生成三日期联立，不唯一的数据记录
   如要检查不唯一数据是否值相等，则可用'3_DATE_唯一性检查.csv，提取出在不同表中的数据，用原表merge，再去重，若去重前后数据多了，
   则找出重复的
    
    '''
    
    #读取12表
    balance_sheet_Bank,balance_sheet_Securities, balance_sheet_Insurance,balance_sheet_GB,\
    cashFlow_sheet_Bank,cashFlow_sheet_Securities,cashFlow_sheet_Insurance,cashFlow_sheet_GB,\
    income_sheet_Bank,income_sheet_Securities,income_sheet_Insurance,income_sheet_GB=loading_12_file()
    
    

    sheet_list=['Balance','Balance','Balance','Balance','cashFlow','cashFlow','cashFlow','cashFlow','Income','Income','Income','Income']

    cate_list=['Bank','Securities','Insurance','GB','Bank','Securities','Insurance','GB','Bank','Securities','Insurance','GB']

    T=[]

    for df,sheet_name,cate_name in zip(df_list,sheet_list,cate_list):
        t=df.groupby(['TICKER_SYMBOL','PUBLISH_DATE','END_DATE_REP','END_DATE']).size()
        t_new=t[t!=1].reset_index()
        t_new['SHEET']=sheet_name
        t_new['CATEGORY']=cate_name
        T.append(t_new)

    pd.concat(T).to_csv(file_path+'3_DATE_唯一性检查.csv',index=False)
    
    pass


def date_time_index(df):
    '''
    datatime函数排序的时候加上了index
    '''
    df['END_DATE']=pd.to_datetime(df['END_DATE'],format='%Y/%m/%d')
    df['PUBLISH_DATE']=pd.to_datetime(df['PUBLISH_DATE'],format='%Y/%m/%d')
    df['END_DATE_REP']=pd.to_datetime(df['END_DATE_REP'],format='%Y/%m/%d')
    df.sort_values(by=['TICKER_SYMBOL','END_DATE','PUBLISH_DATE','END_DATE_REP','INDEX'],inplace=True)
    
    return df


def fill_nan_PIT():
    
    '''
    将12表的point in time 最新数据缺失值补全
    '''

    balance_sheet_Bank_old,balance_sheet_Securities_old,balance_sheet_Insurance_old,balance_sheet_GB_old,\
    cashFlow_sheet_Bank_old,cashFlow_sheet_Securities_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_GB_old,\
    income_sheet_Bank_old,income_sheet_Securities_old,income_sheet_Insurance_old,income_sheet_GB_old=loading_12_file()

    balance_sheet_Bank_old['INDEX']=[x for x in range(len(balance_sheet_Bank_old))]
    balance_sheet_Bank_old




    files= os.listdir(split_file_path) 

    for file in files:
        feature = pd.read_csv(split_file_path+file)
        print(file)
        feature['INDEX']=[x for x in range(len(feature))]
        T=date_time_index(feature)
        T.columns

        common_col=['PARTY_ID', 'TICKER_SYMBOL', 'EXCHANGE_CD', 'PUBLISH_DATE',
                 'END_DATE_REP', 'END_DATE', 'REPORT_TYPE', 'FISCAL_PERIOD',
                 'MERGED_FLAG','INDEX']
        rest_col=set(T.columns)-set(common_col)
        rest_col

        mid_1=T[common_col].drop_duplicates(subset=['TICKER_SYMBOL','END_DATE'])
        mid_1['END_DATE']=mid_1['END_DATE'].apply(lambda x:datetime.datetime.strftime(x,'%Y-%m-%d'))
        ticker_date=zip(mid_1.TICKER_SYMBOL,mid_1.END_DATE)
        ticker_date


        start_time=time.time()

        Ticker=[]
        Enddate=[]
        Col=[]
        Value=[]

        for (a,b) in ticker_date:

            #print(a,b)
            mid_2=T[(T.TICKER_SYMBOL==a)&(T.END_DATE==b)]   
            mid_2
            for col in rest_col:
                #print(col)
                #common_col.extend(rest_col)
                #mid_2[common_col]

                ##当长度为1，直接跳出；当长度不是1，但全是nan或者全0，全部跳出;然后常规判断，当最新消息为空或零，依次往上找，直到找到最近的有值的
                if (len(mid_2[col].values)==1):
                    continue
                elif ((np.isnan(mid_2[col].unique()[0])or(mid_2[col].unique()[0]==0))&(len(mid_2[col].unique())==1)):
                    continue  
                else:
                    i=len(mid_2[col])-1

                    while((np.isnan(mid_2[col].values[i])or(mid_2[col].values[i]==0))&(i>0)):
                        i-=1

                    if i==(len(mid_2[col])-1):
                        continue
                    else:
                        #print(1)
                        Index=T[(T.TICKER_SYMBOL==a)&(T.END_DATE==b)].index[-1]
                        T.loc[[Index],col]=mid_2[col].values[i]
                        Ticker.append(a)
                        Enddate.append(b)
                        Col.append(col)
                        Value.append(mid_2[col].values[i])

                    #T[(T.TICKER_SYMBOL==1)&(T.END_DATE=='2008-12-31')].loc[Index,'AA']
        DataFrame({'ticker': Ticker,'enddate':Enddate,'col':Col,'value': Value}).to_csv(file_path+'fix_bug/record_%s.csv'%file[0:-4],index=False)
        T.to_csv(file_path+'fix_bug/%s_new.csv'%file[0:-4],index=False)
        print('dealing 1st sheet use time:{}\n'.format(time.time()-start_time))
        del T
        gc.collect()

        
